# Chapter 6: Multi-Agent Workflows with Pydantic Graph and Pydantic AI

In real-world AI applications, a single agent is often not enough to handle complex tasks efficiently. Instead, multiple agents can collaborate in a multi-agent workflow, where each agent is responsible for a specific function.

Pydantic Graph provides a structured approach to defining and managing multi-agent workflows, ensuring data flow between agents in a well-defined dependency graph. This chapter will cover:

- Understanding Pydantic Graph and its role in multi-agent AI workflows.
- Building a multi-agent system using Pydantic AI and Pydantic Graph.
- Executing and visualizing the workflow with Mermaid code generation.

Before we start, we need to set up the environment.

In [1]:
import nest_asyncio
nest_asyncio.apply()

from pydantic_ai import Agent, RunContext
import os
import dotenv

dotenv.load_dotenv()

# Set your Google API key
os.environ["GOOGLE_API_KEY"] = os.getenv("GEMINI_API_KEY")

## 1. What is Pydantic Graph?

Pydantic Graph is a graph-based execution framework that allows you to define workflows in a structured way using nodes and edges.

Instead of running tasks sequentially (like in normal Python scripts), you define a graph of tasks (nodes), where each node represents a step in your workflow, and edges define the flow of execution.

This structure is powerful for AI workflows, where tasks like data collection, processing, AI generation, and review need to be modular and interconnected.

## 2. Basics of Pydantic Graph

A Pydantic Graph consists of:

- Nodes (Tasks) → Individual steps in your workflow.
- State → A global object that holds data and evolves as nodes process it.
- Edges (Transitions) → Define which node executes next.
- Graph → The full execution pipeline connecting all nodes.




Let’s visualize a basic example where we fetch user input, process it, and output a result.

```mermaid
graph TD
    A[Get User Input] --> B[Process Data]
    B --> C[Generate Output]
    C --> D[End]
```

Each box (A, B, C) represents a node (step in the workflow). The arrows show the flow of execution.

## 3. Defining the Core Data Structure: State

The State class is a shared object that carries data between nodes.

In [2]:
from dataclasses import dataclass, field

@dataclass
class State:
    user_name: str = ""
    user_interests: list[str] = field(default_factory=list)
    recommended_articles: list[str] = field(default_factory=list)
    email_content: str = ""

- user_name → Stores the user’s name.
- user_interests → Stores topics the user is interested in.
- recommended_articles → Holds articles generated by AI.
- email_content → The final newsletter email.

This State object will be updated as nodes execute.

## 4. Defining the Nodes (A Step in the Workflow)

A node is a class that extends `BaseNode` and has a `run()` method, which performs a task and returns the next node to execute.

Let’s define a simple node that asks for user input.

In [4]:
from pydantic_graph import BaseNode, GraphRunContext, End

@dataclass
class GetUserInfo(BaseNode[State]):
    async def run(self, ctx: GraphRunContext[State]) -> "ProcessUserInfo":
        name = input("Enter your name: ")
        interests = input("Enter your interests (comma-separated): ").split(",")
        
        ctx.state.user_name = name
        ctx.state.user_interests = interests
        
        return ProcessUserInfo()  # Moves to the next step

Explanation:

- The node gets user input (name & interests).
- It updates the state object with the values.
- It returns the next node (`ProcessUserInfo`), defining the execution flow.


## 5. Connecting Nodes to Form a Workflow

Now, let’s create another node that processes user interests and fetches recommended articles.

In [5]:
@dataclass
class ProcessUserInfo(BaseNode[State]):
    async def run(self, ctx: GraphRunContext[State]) -> "GenerateEmail":
        interests = ctx.state.user_interests
        
        # Simulate AI fetching articles
        ctx.state.recommended_articles = [f"Latest news on {topic}" for topic in interests]
        
        return GenerateEmail()

Explanation:

- It processes user interests.
- It generates a list of articles based on those interests.
- It returns the next node (`GenerateEmail`).



# 6. Final Node: Generating an Email

In [15]:
from pydantic import BaseModel

class Article(BaseModel):
    title: str
    subtitle: str
    content: str

article_agent = Agent(
    'google-gla:gemini-1.5-flash',
    system_prompt='You are a newsletter writer. You have to write a newsletter email for a user about the topics they are interested in.',
    result_type=Article  # Enforcing the structured output
)

In [29]:
@dataclass
class GenerateEmail(BaseNode[State]):
    async def run(self, ctx: GraphRunContext[State]) -> "End":
        query = f"Generate a newsletter email content in markdown format for the user name called {ctx.state.user_name} about the following topics: {', '.join(ctx.state.user_interests)}"
        ctx.state.email_content = await article_agent.run(query)
        
        return End(ctx.state.email_content)

This is the final step, where:

- The newsletter email is generated.
- The workflow ends.

## 7. Creating the Pydantic Graph

Now that we have all our nodes, let’s define our Pydantic Graph.

In [30]:
from pydantic_graph import Graph

newsletter_graph = Graph(
    nodes=[GetUserInfo, ProcessUserInfo, GenerateEmail]
)

## 8. Executing the Workflow

We now execute the graph-based workflow.

In [31]:
import asyncio
from pydantic_graph import GraphRunContext

async def main():
    state = State()
    ctx = GraphRunContext(state=state, deps={})
    result = await newsletter_graph.run(GetUserInfo(), state=state)

    print("Final Email Output:")
    print("Title: ", result.output.data.title)
    print("Subtitle: ", result.output.data.subtitle)
    print("Content: ", result.output.data.content)
asyncio.run(main())

Final Email Output:
Title:  Reading Nook Newsletter
Subtitle:  Your Monthly Reading Newsletter
Content:  ## Reading Nook: Your Monthly Dose of Literary Delights

Hey ssyok,

Welcome to your monthly Reading Nook! This month, we're diving into the world of books with recommendations, insights, and tips to enhance your reading journey.

**Featured Reads:**

*   **Title:** "The Name of the Wind"
    *   **Author:** Patrick Rothfuss
    *   **Genre:** Fantasy
    *   **Why we love it:** A captivating tale of magic, mystery, and self-discovery. Perfect for fantasy enthusiasts.

*   **Title:** "To Kill a Mockingbird"
    *   **Author:** Harper Lee
    *   **Genre:** Classic Literature
    *   **Why we love it:** A timeless novel exploring themes of justice, compassion, and childhood innocence.  A must-read for everyone.

**Reading Tip of the Month:**

Try the "30-minute rule." Commit to reading for just 30 minutes each day, even if you only have a small amount of time.  It's amazing how quick

This will:

- Start the workflow from GetUserInfo.
- Move through nodes automatically.
- Generate a personalized email.

## 9. Visualizing the Workflow with Mermaid

You can generate a Mermaid graph with:

In [10]:
newsletter_graph.mermaid_code(start_node=GetUserInfo)

'---\ntitle: newsletter_graph\n---\nstateDiagram-v2\n  [*] --> GetUserInfo\n  GetUserInfo --> ProcessUserInfo\n  ProcessUserInfo --> GenerateEmail\n  GenerateEmail --> [*]'

### Workflow Diagram
```mermaid
graph TD
    A[Get User Input] --> B[Process Data]
    B --> C[Generate Output]
    C --> D[End]
```

## Conclusion

In this chapter, we explored the powerful combination of Pydantic Graph and Pydantic AI for building multi-agent workflows. By structuring our application as a directed graph of specialized nodes, we created a modular, maintainable system where:
- Each node represents a discrete step in our workflow
- Data flows through the system in a well-defined manner via the State object
- Dependencies between steps are explicitly modeled as graph edges

This approach offers several advantages over traditional sequential programming:
- Modularity: Each component has a single responsibility, making the system easier to understand and maintain
- Flexibility: Nodes can be rearranged or replaced without disrupting the entire workflow
- Visualization: The graph structure can be visualized with Mermaid, providing clear documentation
- Scalability: Complex workflows can be built by composing simpler components

The newsletter generation example demonstrated how to combine user input, data processing, and AI-powered content generation in a cohesive workflow. This pattern can be extended to more complex scenarios involving multiple agents, each with specialized capabilities.

As AI systems grow more sophisticated, the ability to orchestrate multiple agents in structured workflows will become increasingly important. Pydantic Graph provides a solid foundation for building these next-generation AI applications.